In [4]:
from astropy.table import Table
from matplotlib import pyplot as plt
%matplotlib inline
import os
import numpy as np
from astropy.io.ascii import masked
from astropy.coordinates import Angle
from astropy.io import ascii
import glob
from astropy.io import fits
import wget
import matplotlib.image as mpimg
from astropy.wcs import WCS
from scipy.stats import scoreatpercentile
from astropy.visualization import simple_norm
from reproject import reproject_interp
import sys
from IPython.display import clear_output
from photutils.detection import DAOStarFinder
from astropy.stats import sigma_clipped_stats
from photutils.aperture import CircularAperture
from astropy.visualization import SqrtStretch
from astropy.visualization import ImageNormalize
from astropy.visualization import LogStretch
from astropy.wcs import WCS
import astropy.units as u
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import Angle
from scipy import stats
from astropy.visualization import MinMaxInterval
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from matplotlib import pyplot as plt
from matplotlib import colors
from astropy.stats import gaussian_sigma_to_fwhm

from astropy.nddata import CCDData
import warnings
warnings.filterwarnings('ignore')

mycolors = plt.rcParams['axes.prop_cycle'].by_key()['color']

from photutils.isophote import EllipseGeometry
from photutils.aperture import EllipticalAperture

#define an empty dictionary that will contain the EllipseGeometry instance
geometry = {}
initparams = {}
#initialize dictionary for half-light radii
rhalfpix = {}
rhalfasec = {}
from photutils.isophote import Ellipse
from photutils import aperture_photometry
#initialize dictionary for ellipse fitting
ellipse = {}
isolist = {}

In [5]:
#Set the home path

os.environ['HOME'] ='C:/Users/USER/Documents/GitHub'
homedir = os.getenv("HOME")
tabledir = homedir+'/Virgo/tables'
plotdir = homedir+'/Virgo/plots'
datadir = homedir+'/HTML-building/galaxy'

In [6]:
def find_files(destination_folder, partial_name):
    matching_files = []

    for root, dirs, files in os.walk(destination_folder):
        for file in files:
            if partial_name.lower() in file.lower():
                matching_files.append(os.path.join(root, file))

    return matching_files 
def aperture(data):
        '''
        # rmax is max radius to measure ellipse
        # could cut this off based on SNR
        # or could cut this off based on enclosed flux?
        # or could cut off based on image dimension, and do the cutting afterward
        
        #rmax = 2.5*self.sma
        '''
        # rmax is set according to the image dimensions
        # look for where the semi-major axis hits the edge of the image
        # could by on side (limited by x range) or on top/bottom (limited by y range)
        # 
        yimage_max, ximage_max = data.shape
        
        rmax = np.min([(ximage_max - x0)/abs(np.cos(PAN)),\
                        (yimage_max - y0)/abs(np.sin(PAN))])
        index = np.arange(80)
        apertures = (index+1)*.5*.7*(1+(index+1)*.1)
        # cut off apertures at edge of image
        apertures_a = apertures[apertures < rmax]
        return apertures_a
def measure_phot(data, apertures_a):
        apertures_b = (1.-EPLI)*apertures_a
        area = np.pi*apertures_a*apertures_b # area of each ellipse

        flux1 = np.zeros(len(apertures_a),'f')
        allellipses = []
        for i in range(len(apertures_a)):
            # EllipticalAperture takes rotation angle in radians, CCW from +x axis
            ap = EllipticalAperture((x0, y0),apertures_a[i],apertures_b[i],PAN)#,ai,bi,theta) for ai,bi in zip(a,b)]
            allellipses.append(ap)
                    # subpixel is the method used by Source Extractor
            phot_table1 = aperture_photometry(data, ap, method = 'subpixel', 
                                                    subpixels=5)
            
            flux1[i] = phot_table1['aperture_sum'][0]
            if np.isnan(flux1[i]):
                break
        # first aperture is calculated differently
        sb1 = np.zeros(len(apertures_a),'f')

        sb1[0] = flux1[0]/area[0]
        # outer apertures need flux from inner aperture subtracted
        for i in range(1,len(area)):
            sb1[i] = (flux1[i] - flux1[i-1])/(area[i]-area[i-1])    
        valid_indices = ~np.isnan(sb1) & ~np.isnan(flux1) & (sb1 != 0) & (flux1 != 0)
        sb1 = sb1[valid_indices]
        flux1 = flux1[valid_indices]
        apertures_a = apertures_a[valid_indices]
        flux1_err = np.zeros(len(apertures_a),'f')
        sb1_err = np.zeros(len(apertures_a),'f')
        print('Number of apertures for',wave ,'microns band = ',len(apertures_a))
        # Find the maximum values
        sb1max = np.max(sb1)
        flux1max = np.max(flux1)
        apertures_ahalf1 = np.max(apertures_a) / 2
    
        return sb1, flux1, sb1max, flux1max, apertures_ahalf1, flux1_err, sb1_err, apertures_a

def plot_profilesbgr(sb1, sb1_err, flux1, flux1_err, ap1,sb2, sb2_err, flux2, flux2_err, ap2 ,sb3, sb3_err, flux3, flux3_err, ap3,flux_yscale='log'):
        ''' enclosed flux and surface brightness profiles, save figure '''
        
        plt.close("all")        
        plt.figure(figsize=(10,4))
        plt.subplots_adjust(wspace=.3)
        plt.subplot(2,4,1)
        plt.errorbar(ap1,flux1,
                     flux1_err,fmt='b.')
        plt.title('B-band (70 microns)')
        plt.ylabel('Flux')
        if flux_yscale=='log':
            plt.gca().set_yscale('log')
        
        plt.subplot(2,4,2)
        plt.errorbar(ap2,flux2,
                     flux2_err,fmt='g.')
        plt.title('G-band (100 microns)')
        if flux_yscale=='log':
            plt.gca().set_yscale('log')
        plt.subplot(2,4,3)
        plt.errorbar(ap3,flux3,
                     flux3_err,fmt='r.')
        plt.title('R-band (160 microns)')
        if flux_yscale=='log':
            plt.gca().set_yscale('log')
        
        plt.subplot(2,4,4)
        label1='Blue'
        label2='Green'
        label3='Red'
        plt.errorbar(ap1, flux1/np.max(flux1), 
                     flux1_err/np.max(flux1_err), fmt='b.',label=label1)
        plt.errorbar(ap2, flux2/np.max(flux2), 
                     flux2_err/np.max(flux2_err), fmt='g.',label=label2)
        plt.errorbar(ap3, flux3/np.max(flux3), 
                     flux3_err/np.max(flux3_err), fmt='r.',label=label3)
        plt.title('Normalized Curves')
        if flux_yscale=='log':
            plt.gca().set_yscale('log')
        plt.legend()
        
        plt.subplot(2,4,5)
        plt.errorbar(ap1,sb1,sb1_err,fmt='b.')
        plt.ylabel('Surface Brightness')
        plt.xlabel('Semi-Major Axis [px]')
        plt.gca().set_yscale('log')
        
        plt.subplot(2,4,6)
        plt.errorbar(ap2,sb2,
                     sb2_err,fmt='g.')
        plt.xlabel('Semi-Major Axis [px]')
        plt.gca().set_yscale('log')

        plt.subplot(2,4,7)
        plt.errorbar(ap3,sb3,
                     sb3_err,fmt='r.')
        plt.xlabel('Semi-Major Axis [px]')
        plt.gca().set_yscale('log')
    
        plt.subplot(2,4,8)
        plt.errorbar(ap1, sb1/np.max(sb1), 
                     sb1_err/np.max(sb1_err), fmt='b.')
        plt.errorbar(ap2, sb2/np.max(sb2), 
                     sb2_err/np.max(sb2_err), fmt='g.')
        plt.errorbar(ap3, sb3/np.max(sb3), 
                     sb3_err/np.max(sb3_err), fmt='r.')
        plt.gca().set_yscale('log')
        plt.xlabel('Semi-Major Axis [px]')
        
        plt.savefig(datadir+'/photometry/'+VFID+'-'+galaxy_name+'.png', dpi=150)
        plt.close()
        
def plot_profilesbr(sb1, sb1_err, flux1, flux1_err, ap1,sb3, sb3_err, flux3, flux3_err, ap3,flux_yscale='log'):
        ''' enclosed flux and surface brightness profiles, save figure '''
        
        plt.close("all")        
        plt.figure(figsize=(10,4))
        plt.subplots_adjust(wspace=.3)
        
        plt.subplot(2,3,1)
        plt.errorbar(ap1,flux1,
                     flux1_err,fmt='b.')
        plt.title('B-band (70 microns)')
        plt.ylabel('Flux')
        if flux_yscale=='log':
            plt.gca().set_yscale('log')
        
        plt.subplot(2,3,2)
        plt.errorbar(ap3,flux3,
                     flux3_err,fmt='r.')
        plt.title('R-band (160 microns)')
        if flux_yscale=='log':
            plt.gca().set_yscale('log')
        
        plt.subplot(2,3,3)
        label1='Blue'
        label2='Red'
        plt.errorbar(ap1, flux1/np.max(flux1), 
                     flux1_err/np.max(flux1_err), fmt='b.',label=label1)
        plt.errorbar(ap3, flux3/np.max(flux3), 
                     flux3_err/np.max(flux3_err), fmt='r.',label=label2)
        plt.title('Normalized Curves')
        if flux_yscale=='log':
            plt.gca().set_yscale('log')
        plt.legend()
        
        plt.subplot(2,3,4)
        plt.errorbar(ap1,sb1,sb1_err,fmt='b.')
        plt.ylabel('Surface Brightness')
        plt.xlabel('Semi-Major Axis [px]')
        plt.gca().set_yscale('log')
        

        plt.subplot(2,3,5)
        plt.errorbar(ap3,sb3,
                     sb3_err,fmt='r.')
        plt.xlabel('Semi-Major Axis [px]')
        plt.gca().set_yscale('log')
    
        plt.subplot(2,3,6)
        plt.errorbar(ap1, sb1/np.max(sb1), 
                     sb1_err/np.max(sb1_err), fmt='b.')
        plt.errorbar(ap3, sb3/np.max(sb3), 
                     sb3_err/np.max(sb3_err), fmt='r.')
        plt.gca().set_yscale('log')
        plt.xlabel('Semi-Major Axis [px]')
        
        plt.savefig(datadir+'/photometry/'+VFID+'-'+galaxy_name+'.png', dpi=150)
        plt.close()
def plot_profilesgr(sb2, sb2_err, flux2, flux2_err, ap2,sb3, sb3_err, flux3, flux3_err, ap3,flux_yscale='log'):
        ''' enclosed flux and surface brightness profiles, save figure '''
        
        plt.close("all")        
        plt.figure(figsize=(10,4))
        plt.subplots_adjust(wspace=.3)
        
        plt.subplot(2,3,1)
        plt.errorbar(ap2,flux2,
                     flux2_err,fmt='g.')
        plt.title('G-band (100 microns)')
        plt.ylabel('Flux')
        if flux_yscale=='log':
            plt.gca().set_yscale('log')
        
        plt.subplot(2,3,2)
        plt.errorbar(ap3,flux3,
                     flux3_err,fmt='r.')
        plt.title('R-band (160 microns)')
        if flux_yscale=='log':
            plt.gca().set_yscale('log')
        
        plt.subplot(2,3,3)
        label1='Green'
        label2='Red'
        plt.errorbar(ap2, flux2/np.max(flux2), 
                     flux2_err/np.max(flux2_err), fmt='g.',label=label1)
        plt.errorbar(ap3, flux3/np.max(flux3), 
                     flux3_err/np.max(flux3_err), fmt='r.',label=label2)
        plt.title('Normalized Curves')
        if flux_yscale=='log':
            plt.gca().set_yscale('log')
        plt.legend()
        
        plt.subplot(2,3,4)
        plt.errorbar(ap2,sb2,sb2_err,fmt='g.')
        plt.ylabel('Surface Brightness')
        plt.xlabel('Semi-Major Axis [px]')
        plt.gca().set_yscale('log')
        

        plt.subplot(2,3,5)
        plt.errorbar(ap3,sb3,
                     sb3_err,fmt='r.')
        plt.xlabel('Semi-Major Axis [px]')
        plt.gca().set_yscale('log')
    
        plt.subplot(2,3,6)
        plt.errorbar(ap2, sb2/np.max(sb2), 
                     sb2_err/np.max(sb2_err), fmt='g.')
        plt.errorbar(ap3, sb3/np.max(sb3), 
                     sb3_err/np.max(sb3_err), fmt='r.')
        plt.gca().set_yscale('log')
        plt.xlabel('Semi-Major Axis [px]')
        
        plt.savefig(datadir+'/photometry/'+VFID+'-'+galaxy_name+'.png', dpi=150)
        plt.close()

In [14]:
csv_file = tabledir+'/Photometrytesting.csv'
galaxy = Table.read(csv_file)
# Determine the length of the galaxy
galaxy_length = len(galaxy)
# Assign zeros to the variables
sb3max = np.zeros(galaxy_length, dtype='f')
flux3max = np.zeros(galaxy_length, dtype='f')
apertures_ahalf3 = np.zeros(galaxy_length, dtype='f')
sb2max = np.zeros(galaxy_length, dtype='f')
flux2max = np.zeros(galaxy_length, dtype='f')
apertures_ahalf2 = np.zeros(galaxy_length, dtype='f')
sb1max = np.zeros(galaxy_length, dtype='f')
flux1max = np.zeros(galaxy_length, dtype='f')
apertures_ahalf1 = np.zeros(galaxy_length, dtype='f')
for i in range(len(galaxy)):
    galaxy_name = str(galaxy['GALAXY'][i])
    RA = galaxy['RA_MOMENT'][i]
    DEC = galaxy['DEC_MOMENT'][i]
    EPLI = galaxy['BA_MOMENT'][i]
    PAN = (galaxy['PA_MOMENT'][i] + 90) * np.pi/180
    SMAO = galaxy['SMA_MOMENT'][i]
    path =  datadir+'/pipeline/'+galaxy_name
    VFID = str(galaxy['VFID'][i])
    if os.path.exists(path):
        hasblue = False
        hasgreen = False
        hasred = False
        destination_folder = path+'\\HPPUNIMAPR'
        partial_name = 'hpacs_25HPPUNIMAPR'
        found_files3 = find_files(destination_folder, partial_name)
        if found_files3:
            found_file3 = found_files3[0]
            placeholder = fits.open(found_file3)
            pixscale = placeholder[0].header['PIXSIZE']
            wave = placeholder[0].header['WAVELNTH']
            SMAN = (SMAO * 0.262) / pixscale
            data, header = fits.getdata(found_file3, header=True)
            wcs = WCS(header)
            x0, y0 = wcs.all_world2pix(RA, DEC, 0)
            initparams[galaxy_name] = {}
            initparams[galaxy_name]['xcen'] = x0
            initparams[galaxy_name]['ycen'] = y0
            initparams[galaxy_name]['sma'] = SMAN
            initparams[galaxy_name]['ellip'] = EPLI
            initparams[galaxy_name]['PA'] = PAN
            aperture_a = aperture(data)
            hasred = True
            sb3, flux3, sb3max[i], flux3max[i], apertures_ahalf3[i], sb3_err, flux3_err, n_ap3 = measure_phot(data, aperture_a)
        destination_folder = path+'\\HPPUNIMAPB'
        partial_name = 'hpacs_25HPPUNIMAPB'
        found_files1 = find_files(destination_folder, partial_name)
        if found_files1:
            found_file1 = found_files1[0]
            placeholder = fits.open(found_file1)
            pixscale = placeholder[0].header['PIXSIZE']
            wave = placeholder[0].header['WAVELNTH']
            SMAN = (SMAO * 0.262) / pixscale
            data, header = fits.getdata(found_file1, header=True)
            wcs = WCS(header)
            x0, y0 = wcs.all_world2pix(RA, DEC, 0)
            initparams[galaxy_name] = {}
            initparams[galaxy_name]['xcen'] = x0
            initparams[galaxy_name]['ycen'] = y0
            initparams[galaxy_name]['sma'] = SMAN
            initparams[galaxy_name]['ellip'] = EPLI
            initparams[galaxy_name]['PA'] = PAN
            hasblue = True
            sb1, flux1, sb1max[i], flux1max[i], apertures_ahalf1[i], sb1_err, flux1_err, n_ap1 = measure_phot(data, aperture_a)            
        destination_folder = path+'\\HPPUNIMAPG'
        partial_name = 'hpacs_25HPPUNIMAPB'
        found_files2 = find_files(destination_folder, partial_name)
        if found_files2:
            found_file2 = found_files2[0]
            placeholder = fits.open(found_file2)
            pixscale = placeholder[0].header['PIXSIZE']
            wave = placeholder[0].header['WAVELNTH']
            SMAN = (SMAO * 0.262) / pixscale
            data, header = fits.getdata(found_file2, header=True)
            wcs = WCS(header)
            x0, y0 = wcs.all_world2pix(RA, DEC, 0)
            initparams[galaxy_name] = {}
            initparams[galaxy_name]['xcen'] = x0
            initparams[galaxy_name]['ycen'] = y0
            initparams[galaxy_name]['sma'] = SMAN
            initparams[galaxy_name]['ellip'] = EPLI
            initparams[galaxy_name]['PA'] = PAN
            hasgreen = True
            sb2, flux2, sb2max[i], flux2max[i], apertures_ahalf2[i], sb2_err, flux2_err, n_ap2 = measure_phot(data, aperture_a)    
        if hasblue == True & hasgreen == True:
            plot_profilesbgr(sb1, sb1_err, flux1, flux1_err, n_ap1,sb2, sb2_err, flux2, flux2_err, n_ap2,sb3, sb3_err, flux3, flux3_err, n_ap3,flux_yscale='log')
        if hasblue and not hasgreen:
            plot_profilesbr(sb1, sb1_err, flux1, flux1_err, n_ap1,sb3, sb3_err, flux3, flux3_err, n_ap3,flux_yscale='log')  
        if not hasblue and hasgreen:
            plot_profilesgr(sb2, sb2_err, flux2, flux2_err, n_ap2,sb3, sb3_err, flux3, flux3_err, n_ap3,flux_yscale='log') 
    else:
        print(f"Galaxy not found: {VFID}")
        print(galaxy_name)
print('done')

Number of apertures for 160.0 microns band =  30
Number of apertures for 70.0 microns band =  40
Number of apertures for 100.0 microns band =  40
Number of apertures for 160.0 microns band =  33
Number of apertures for 70.0 microns band =  37
Number of apertures for 100.0 microns band =  38
Number of apertures for 160.0 microns band =  59
Number of apertures for 70.0 microns band =  72
Number of apertures for 100.0 microns band =  72
Number of apertures for 160.0 microns band =  41
Number of apertures for 70.0 microns band =  45
Number of apertures for 100.0 microns band =  45
Number of apertures for 160.0 microns band =  57
Number of apertures for 70.0 microns band =  61
Number of apertures for 100.0 microns band =  57
Number of apertures for 160.0 microns band =  57
Number of apertures for 100.0 microns band =  62
Number of apertures for 160.0 microns band =  51
Number of apertures for 100.0 microns band =  71
Number of apertures for 160.0 microns band =  39
Number of apertures for 1

In [42]:
galaxy = Table.read(tabledir+'/Photometrytesting.csv')
herschel = Table.read(tabledir+'/Herschelstuff.csv')
for i in range(len(galaxy)):
    galaxy_name = str(galaxy['GALAXY'][i])
    path = datadir + '\\pipeline\\' + galaxy_name
    VFID = str(galaxy['VFID'][i])
    RA = str(galaxy['RA_MOMENT'][i])
    DEC = str(galaxy['DEC_MOMENT'][i])
    filepath = datadir+'/html/'+VFID+'-'+galaxy_name+'.html'
    blueflag = herschel['70micronsflag'][i]
    greenflag = herschel['100micronsflag'][i]
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    # Write the HTML code line by line
    with open(filepath, "w") as html:
        html.write('<html><body>\n')
        html.write('<title>Photometry data</title>\n')
        html.write('<style type="text/css">\n')
        html.write('table, td, th {padding: 5px; text-align: center; border: 2px solid black;}\n')
        html.write('p {display: inline-block;;}\n')
        html.write('</style>\n')
        html.write('<table width="100%"><tr><th>VFID</th><th>Name</th><th>Legacy image</th><th>Herschel-UnimapBlue (70microns)</th><th>Herschel-UnimapBlue (100microns)</th><th>Herschel-UnimapRed (160microns)</th><th>RA</th><th>DEC</th></tr></p>')
        html.write('<tr><td>'+VFID+'</td><td>'+galaxy_name+'</td><td><a href="../png/'+VFID+'-'+galaxy_name+'-LS.jpg"><img src="../png/'+VFID+'-'+galaxy_name+'-LS.jpg" alt="No LS data.jpg" height="auto" width="100%"></a></td><td><a href="../png/'+VFID+'-'+galaxy_name+'blue.png"><img src="../png/'+VFID+'-'+galaxy_name+'blue.png" alt="Missing file 70microns.jpg" height="auto" width="100%"></a></td><td><a href="../png/'+VFID+'-'+galaxy_name+'green.png"><img src="../png/'+VFID+'-'+galaxy_name+'green.png" alt="Missing file 100microns.jpg" height="auto" width="100%"></a></td><td><a href="../png/'+VFID+'-'+galaxy_name+'red.png"><img src="../png/'+VFID+'-'+galaxy_name+'red.png" alt="Missing file 160microns.jpg" height="auto" width="100%"></a></td><td>'+RA+'</td><td>'+DEC+'</td></tr> \n')
        html.write('</table>\n')
        html.write('<h2>Photometry results</h2>')
        html.write('<table width="100%"')
        html.write('<tr><th>Photometry poster</th></tr></p>')
        html.write('<tr><td><a href="../photometry/'+VFID+'-'+galaxy_name+'.png"><img src="../photometry/'+VFID+'-'+galaxy_name+'.png" alt="No Photometry data.png" height="auto" width="100%"></td></tr>')
        html.write('</table>')
        html.write('<h2>Other photometry outputs</h2>')
        html.write('<table width="100%">')
        if blueflag and greenflag:
            html.write('<tr><th>Blue flux max</th><th>Blue Surface brightness max</th><th>Blue half flux radius <br> (pix)</th><th>Green flux max</th><th>Green Surface brightness max</th><th>Green half flux radius <br> (pix)</th><th>Red flux max</th><th>Red Surface brightness max</th><th> Red half flux radius <br> (pix)</th></tr></p>')
            html.write('<tr><td>'+str(flux1max[i])+'</td><td>'+str(sb1max[i])+'</td><td>'+str(apertures_ahalf1[i])+'</td><td>'+str(flux2max[i])+'</td><td>'+str(sb2max[i])+'</td><td>'+str(apertures_ahalf2[i])+'</td><td>'+str(flux3max[i])+'</td><td>'+str(sb3max[i])+'</td><td>'+str(apertures_ahalf3[i])+'</td></tr>')
        if blueflag and not greenflag:
            html.write('<tr><th>Blue flux max</th><th>Blue Surface brightness max</th><th>Blue half flux radius <br> (pix)</th><th>Red flux max</th><th>Red Surface brightness max</th><th> Red half flux radius <br> (pix)</th></tr></p>')
            html.write('<tr><td>'+str(flux1max[i])+'</td><td>'+str(sb1max[i])+'</td><td>'+str(apertures_ahalf1[i])+'</td><td>'+str(flux3max[i])+'</td><td>'+str(sb3max[i])+'</td><td>'+str(apertures_ahalf3[i])+'</td></tr>')
        if not blueflag and greenflag:
            html.write('<tr><th>Green flux max</th><th>Green Surface brightness max</th><th>Green half flux radius <br> (pix)</th><th>Red flux max</th><th>Red Surface brightness max</th><th> Red half flux radius <br> (pix)</th></tr></p>')
            html.write('<tr><td>'+str(flux2max[i])+'</td><td>'+str(sb2max[i])+'</td><td>'+str(apertures_ahalf2[i])+'</td><td>'+str(flux3max[i])+'</td><td>'+str(sb3max[i])+'</td><td>'+str(apertures_ahalf3[i])+'</td></tr>')    
        html.write('</table>\n')
        html.write('<br /><br />\n')
        html.write('</html></body>\n')
        html.close()
print("done")


done
